# Пример использования библиотеки gensim для тематического моделирования

Такая полезная теорема Байеса! :)

![comic1](http://imgs.xkcd.com/comics/seashell.png)

In [1]:
from gensim import corpora, models
# corpora - импортирует данные в нужном формате
# models - строит саму модель

C:\Users\Dima\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"


In [2]:
# Импортируем данные в формте UCI Bag of Words (хорошо представляет данные в разреженной матрице частоты слов)
data = corpora.UciCorpus("docword.xkcd.txt", "vocab.xkcd.txt") # docword: номер документа, номер слова, частота. vocab - словарь
dictionary = data.create_dictionary() # в отдельную переменную нужно сохранить словарь из нашей data

In [3]:
# обучение модель
%time ldamodel = models.ldamodel.LdaModel(data, id2word=dictionary, num_topics=5, passes=20, alpha=1.25, eta=1.25)
# id2word - отображения в слова (dictionary), passes - кол. проходов по коллекции, alpha и eta параметры распределения Дирихле

Wall time: 6min 45s


In [4]:
# Сохранение модели в папку пользователя (сохранит сразу несколько файлов: 4)
ldamodel.save("ldamodel_xkcd")

In [5]:
# Загрузка модели из папки пользователя
# так можно несколько раз генерировать модель, чтобы подобрать хорошее начальное приближение, которо определяется по топам слов
ldamodel = models.ldamodel.LdaModel.load("ldamodel_xkcd")

In [6]:
# выводим топ-10 тем по топ-10 слов в каждой топовой теме (print_topics)
for t, top_words in ldamodel.print_topics(num_topics=10, num_words=10):
    print "Topic", t, ":", top_words

Topic 0 : 0.001*"beef" + 0.001*"guy" + 0.001*"december" + 0.001*"senator" + 0.001*"24th" + 0.001*"pizza" + 0.001*"ray" + 0.001*"silence" + 0.001*"raptor" + 0.000*"hat"
Topic 1 : 0.001*"ago" + 0.001*"male" + 0.001*"gliese" + 0.001*"planet" + 0.001*"female" + 0.001*"day" + 0.001*"year" + 0.001*"destroy" + 0.001*"cast" + 0.001*"stage"
Topic 2 : 0.042*"man" + 0.017*"woman" + 0.007*"text" + 0.006*"title" + 0.006*"hat" + 0.004*"computer" + 0.003*"look" + 0.003*"stand" + 0.003*"black" + 0.003*"one"
Topic 3 : 0.004*"narrator" + 0.002*"wait" + 0.001*"within" + 0.001*"peter" + 0.001*"sagal" + 0.001*"one" + 0.001*"nathan" + 0.001*"part" + 0.001*"relation" + 0.001*"accurate"
Topic 4 : 0.015*"person" + 0.011*"text" + 0.010*"title" + 0.009*"guy" + 0.007*"girl" + 0.006*"one" + 0.005*"two" + 0.005*"just" + 0.005*"people" + 0.004*"figure"


In [7]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду (чем меньше тем лучше)
# нужно сравнивать разные модели по данной метрике, т.к. просто число нам ни о чем не говорит
perplexity = ldamodel.log_perplexity(list(data))
print 2**(-perplexity)

360.804448026


In [8]:
perp = ldamodel.bound(data)
2**(-perp/float(87409))

360.80444801619399

In [ ]:
# (Дообучение модели на новых данных) Добавление в модель новых документов, содержащихся в новом корупсе data2
ldamodel.update(data2, passes=10)

In [10]:
# Получение распределения тем для конкретного документа
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.055873850861296932),
 (1, 0.055326177847326789),
 (2, 0.099772848537461101),
 (3, 0.057913350129272484),
 (4, 0.73111377262464272)]

Эти люди не знают про тематические модели:

![comic2](http://imgs.xkcd.com/comics/the_problem_with_wikipedia.png) | ![comic3](http://imgs.xkcd.com/comics/mystery_news.png)